## Homework 02: Unsupervised embedding-based MT
*Note: this homework is based on open materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [2]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [3]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222576022148132),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.893003523349762),
 ('апрель', 0.8729087710380554),
 ('декабрь', 0.8652558326721191),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401416540145874)]

In [5]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096439480781555),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992520570755005),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592805862426758),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256430625915527)]

In [6]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.27579623460769653),
 ('ЖИЗНИВадим', 0.25203439593315125),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.2443528175354004),
 ('2Михаил', 0.23981396853923798),
 ('лексей', 0.23740753531455994),
 ('комплексн', 0.2369515299797058),
 ('персональ', 0.236822247505188)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [7]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [8]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

--2021-03-28 13:05:20--  http://tiny.cc/jfgecz
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/jfgecz [following]
--2021-03-28 13:05:20--  https://tiny.cc/jfgecz
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt [following]
--2021-03-28 13:05:22--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Sa

In [9]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

--2021-03-28 13:05:22--  http://tiny.cc/6zoeez
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/6zoeez [following]
--2021-03-28 13:05:23--  https://tiny.cc/6zoeez
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt [following]
--2021-03-28 13:05:24--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Savi

In [10]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [11]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [19]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression()

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [20]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531404733657837),
 ('июнь', 0.8402308225631714),
 ('март', 0.838577389717102),
 ('сентябрь', 0.8331867456436157),
 ('февраль', 0.8311493396759033),
 ('октябрь', 0.8278170824050903),
 ('ноябрь', 0.8244149684906006),
 ('июль', 0.8228995203971863),
 ('август', 0.8112362027168274),
 ('январь', 0.8022858500480652)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [37]:
from tqdm.notebook import tqdm

In [67]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in tqdm(enumerate(pairs)):
        most_similar = ru_emb.most_similar([mapped_vectors[i]], topn=topn)
        most_similar = list(map(lambda x: x[0], most_similar))
        num_matches += int(ru in most_similar)
    precision_val = num_matches / len(pairs)
    return precision_val

In [68]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [69]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [70]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [71]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [72]:
import numpy as np

In [73]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    u, s, vh = np.linalg.svd(X_train.T.dot(Y_train))
    mapping = u.dot(vh)
    return mapping

In [74]:
W = learn_transform(X_train, Y_train)

In [75]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131373405457),
 ('июнь', 0.8056629300117493),
 ('сентябрь', 0.8055760264396667),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987099885940552),
 ('июль', 0.7946795225143433),
 ('ноябрь', 0.7939635515213013),
 ('август', 0.7938187122344971),
 ('февраль', 0.7923860549926758),
 ('декабрь', 0.7715373039245605)]

In [76]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0it [00:00, ?it/s]

0.6437659033078881


0it [00:00, ?it/s]

0.7989821882951654


Не сильно луше результаты стали. Может быть датасет слишком маленький

## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [77]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2021-03-28 13:43:10--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1,7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1,73M  1,36MB/s    in 1,3s    

2021-03-28 13:43:14 (1,36 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [78]:
!gzip -d ./uk.txt.gz

In [94]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [95]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [96]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
uk_corpus = [' '.join(tokenizer.tokenize(doc.lower())) for doc in uk_corpus]

In [127]:
def translate(sentence):
    def get_ru(ukr):
        try:
            return ru_emb.most_similar([np.matmul(uk_emb[ukr], W)], topn = 1)[0][0]
        except KeyError:
            return '_'.join(["UNK", ukr])
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = [get_ru(token) for token in sentence.split()]
    return " ".join(translated)

In [128]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [130]:
for sent in tqdm(uk_corpus[::10]):
    print('ukr: ', sent, '\n', 'ru: ', translate(sent),'\n')

  0%|          | 0/100 [00:00<?, ?it/s]

ukr:  я вже закінчу коледж , коли ви вернетеся з америки . 
 ru:  мной уже закончу колледж , когда мы прибежишь со америки . 

ukr:  місто бомбардували ворожі літаки . 
 ru:  город бомбили враждебные самолеты . 

ukr:  можливо , я антисоціальний , але це не означає , що я не спілкуюся з людьми . 
 ru:  возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми . 

ukr:  цього ранку випала роса . 
 ru:  этого утра выпала роса . 

ukr:  біда не приходить одна . 
 ru:  беда не приходит одна . 

ukr:  подивися на той дим . 
 ru:  посмотри по тот дым . 

ukr:  я замовив два гамбургера . 
 ru:  мной заказал два гамбургера . 

ukr:  я не хотів нікого образити . 
 ru:  мной не хотел никого обидеть . 

ukr:  гора вкрита снігом . 
 ru:  гора покрыта снегом . 

ukr:  на фотографії в дівчини корона не з золота , а з квітів . 
 ru:  по фотографии во девушки корона не со золота , а со цветов . 

ukr:  у мене є мрія . 
 ru:  во меня То мечта . 

ukr:  я приїхав у японію 

ukr:  « скільки коштує ця носова хусточка ?» — « дев ' яносто п ' ять центів ». 
 ru:  « сколько стоить та носовая косыночка UNK_?» — « девять со двадцать аш со пять центов UNK_». 

ukr:  ранені ведмеді , як правило , дуже небезпечні . 
 ru:  раненные медведи , как правило , очень опасные . 

ukr:  він швидко втомлюється . 
 ru:  он быстро устает . 

ukr:  усі готові . 
 ru:  все готовы . 

ukr:  він скучає по своїй сім ' ї . 
 ru:  он скучает по своей семь со мье . 

ukr:  « дякую », — « на здоров ' я ». 
 ru:  « спасибо UNK_», — « по здоровье со мной UNK_». 

ukr:  я ще не знаю своєї адреси , я певний час буду жити в подруги . 
 ru:  мной ещe не знаю своего адреса , мной определенный момент буду жить во подруги . 

ukr:  амазонка — друга по довжині ріка в світі після ніла . 
 ru:  амазонка — вторая по длине река во мире после трепещущая . 

ukr:  а якщо побачиш тома , передай йому від мене вітання . 
 ru:  а если увидишь тома , передай ему от меня поздравления . 

ukr:  закрий за соб